In [6]:
import grpc
import sys
import inference_pb2_grpc as pb2_grpc
import inference_pb2 as pb2

In [10]:
# Compile proto3 file
# python3 -m grpc_tools.protoc --proto_path=. ./inference.proto --python_out=. --grpc_python_out=.

In [11]:
class TestClient(object):
    """
    Client for testing inference gRPC
    """

    def __init__(self):
        #self.host = '3.139.238.241'
        self.host = 'localhost'
        self.server_port = 5125

        # instantiate a channel
        self.channel = grpc.insecure_channel(
            '{}:{}'.format(self.host, self.server_port))

        # bind the client and the server
        self.stub = pb2_grpc.InferenceStub(self.channel)

    def send(self, tx, modelHash, modelInput):
        """
        Client function to call the rpc for inference
        """
        message = pb2.InferenceParameters(tx=tx, modelHash=modelHash, modelInput = modelInput)
        print(f'{message}')
        return self.stub.RunInference(message)

def testLinearModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="LinearTest", modelInput="[[1.0, 2.0, 3.0], [2.5, 3.5, 4.5]]")
    print(f'{result}')

def testVolatilityModel():
    client = TestClient()
    result = client.send(tx = "0x123", modelHash="Volatility", modelInput="[[0.03],[0.05],[0.04056685],[0.03235871],[0.05629578]]")
    print(f'{result}')


In [12]:
testLinearModel()

tx: "0x123"
modelHash: "LinearTest"
modelInput: "[[1.0, 2.0, 3.0], [2.5, 3.5, 4.5]]"

tx: "0x123"
node: "0x12345678910"
value: "[17,26]"



In [13]:
testVolatilityModel()

tx: "0x123"
modelHash: "Volatility"
modelInput: "[[0.03],[0.05],[0.04056685],[0.03235871],[0.05629578]]"

tx: "0x123"
node: "0x12345678910"
value: "0.053176194"

